# FABRIC API Example: Create a Single Node


## Configure the Environment

### Create the FABRIC Proxies

The FABRIC API is used via proxy objects that manage connections to the control framework. 

#### Orchestrator Proxy

Manages requests and queries for FABRIC resources. 

#### Credential Manager Proxy

See the [manage_tokens](./manage_tokens.ipynb) example notebook 




In [ ]:
import os
import json

from fabric_cm.credmgr.credmgr_proxy import CredmgrProxy
from fabric_cf.orchestrator.orchestrator_proxy import OrchestratorProxy

#Create a FABRIC Orchestrator Proxy.
orchestrator_proxy = OrchestratorProxy(orchestrator_host=os.environ['FABRIC_ORCHESTRATOR_HOST'])

#Create a Credential Manager Proxy 
credmgr_proxy = CredmgrProxy(credmgr_host=os.environ['FABRIC_CREDMGR_HOST'])

#Cofigure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()


### Save the Initial Refresh Token

In [ ]:
#Retrieve or set the refresh token (exprires 24 hours after login)
fabric_refresh_token=None
%store -r fabric_refresh_token
if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token

print("Refresh Token: {}".format(fabric_refresh_token))

### Refresh the ID Token

ID Tokens expire one hour after refresh. 

In [ ]:
try:
    refresh_res = credmgr_proxy.refresh(project_name='all', scope='all', refresh_token=fabric_refresh_token)

    fabric_id_token=refresh_res['id_token']
    fabric_refresh_token=refresh_res['refresh_token']
    print("New Refreash Token: {}\n".format(fabric_refresh_token))
    print("New ID Token: {}".format(fabric_id_token))
    %store fabric_refresh_token

except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))
    

## Build Slice Request

<!---
<img src="./figs/AllNodes.png" width="90%"><br>
-->


In [ ]:
import fim.user as fu

# Create Experiment Topology
experiemnt = fu.ExperimentTopology()

slice_name="MySlice"

In [ ]:
help(fu.ExperimentTopology)

### Add Node at RENC
<img src="./figs/SingleNode.png" width="40%"><br>

In [ ]:
myNode = experiemnt.add_node(name='node1', site='RENC')

myNode_capacity = fu.Capacities()
myNode_capacity.set_fields(core=2, ram=16, disk=100)
myNode.set_properties(capacities=node1_capacity, image_type='qcow2', image_ref='default_ubuntu_20')


## Delete Slice

In [ ]:
status, result = orchestrator_proxy.delete(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))